<a href="https://colab.research.google.com/github/ELMohamedAyoub/Deepfake-Content-Detection/blob/master/multimodal%2Bv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install -q datasets
!pip install -q newspaper3k

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 110.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 7.9 MB/s eta 0:00:00


In [20]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mahinaik123/fakeddit-dataset")

print("Path to dataset files:", path)

100%|██████████| 785M/785M [00:08<00:00, 93.1MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/mahinaik123/fakeddit-dataset/versions/1


In [21]:
# @title Cell 2: Load Fakeddit from Kaggle Cache
import pandas as pd
import os
import glob

# 1. Define the path provided by the download output
CACHE_DIR = "/root/.cache/kagglehub/datasets/mahinaik123/fakeddit-dataset/versions/1"

print(f"Searching for dataset files in: {CACHE_DIR}...")

# 2. Find the TSV files
# We look for 'multimodal_test_public.tsv' for a manageable first run.
# If you want the full training set later, change 'test' to 'train'.
search_pattern = os.path.join(CACHE_DIR, "*multimodal_test_public*.tsv")
found_files = glob.glob(search_pattern)

# Fallback if specific file not found (grab any tsv)
if not found_files:
    found_files = glob.glob(os.path.join(CACHE_DIR, "*.tsv"))

if not found_files:
    raise FileNotFoundError(f"No .tsv files found in {CACHE_DIR}")

target_file = found_files[0]
print(f"✅ Found dataset file: {os.path.basename(target_file)}")

# 3. Load Data
df = pd.read_csv(target_file, sep='\t')

# 4. Clean & Standardize Columns
# Fakeddit standard keys: author, clean_title, created_utc, domain, hasImage, id, image_url, linked_submission_id, num_comments, score, subreddit, title, upvote_ratio, 2_way_label, 3_way_label, 6_way_label
desired_cols = ['clean_title', 'image_url', '2_way_label']

# Rename if necessary to match our pipeline (text, image_url, label)
df = df[desired_cols].copy()
df.columns = ['text', 'image_url', 'label']

# 5. Filter for Valid Images
# We only want rows with valid http links
df = df[df['image_url'].notna()]
df = df[df['image_url'].str.startswith('http')]

print(f"✅ Dataset Index Loaded Successfully.")
print(f"Total Samples: {len(df)}")
print(f"Class Distribution:\n{df['label'].value_counts()}")

# Save to local CSV for the next cell
df.to_csv("fakeddit_index_local.csv", index=False)

Searching for dataset files in: /root/.cache/kagglehub/datasets/mahinaik123/fakeddit-dataset/versions/1...
✅ Found dataset file: all_test_public.tsv
✅ Dataset Index Loaded Successfully.
Total Samples: 67099
Class Distribution:
label
0    43687
1    23412
Name: count, dtype: int64


In [22]:
# @title Cell 3: Asynchronous Image Acquisition
import os
import time
import io
import asyncio
import aiohttp
from PIL import Image

# 1. Setup Storage
IMAGE_DIR = "data/images"
os.makedirs(IMAGE_DIR, exist_ok=True)

# 2. Configuration
# For development/testing pipeline, we limit to 2000 images.
# For full training, set this to: len(df)
DOWNLOAD_LIMIT = 2000

print(f"Initializing download pipeline for {DOWNLOAD_LIMIT} samples...")

# Select subset for download
df_subset = df.head(DOWNLOAD_LIMIT).copy()

async def fetch_image(session, url, index, timeout=10):
    """
    Asynchronously fetches an image from a URL, verifies integrity, and saves to disk.
    Returns: (index, success_boolean)
    """
    try:
        async with session.get(url, timeout=timeout) as response:
            if response.status == 200:
                content = await response.read()
                # Verification: Attempt to open as Image
                try:
                    with Image.open(io.BytesIO(content)) as img:
                        img.verify() # check for corruption

                    # Re-open to convert and save (verify closes the file)
                    with Image.open(io.BytesIO(content)) as img:
                        # Convert to RGB to handle PNG/RGBA issues standardizing for CLIP
                        img = img.convert('RGB')
                        save_path = os.path.join(IMAGE_DIR, f"{index}.jpg")
                        img.save(save_path)
                        return index, True
                except Exception:
                    # Image is corrupt or not an image
                    return index, False
            return index, False
    except Exception:
        # Network timeout or 404
        return index, False

async def download_manager(urls, indices):
    """
    Manages the concurrent session.
    """
    # Limit concurrent connections to avoid being flagged as a bot
    connector = aiohttp.TCPConnector(limit=50)
    async with aiohttp.ClientSession(connector=connector) as session:
        tasks = []
        for url, idx in zip(urls, indices):
            tasks.append(fetch_image(session, url, idx))
        return await asyncio.gather(*tasks)

# 3. Execute Download
start_time = time.time()

# In Jupyter/Colab, the event loop is already running, so we await directly
print(f"Downloading images... (Target: {DOWNLOAD_LIMIT})")
results = await download_manager(df_subset['image_url'], df_subset.index)

# 4. Post-Processing & Cleanup
# Filter the dataframe to keep only samples where the image was successfully downloaded
valid_indices = [idx for idx, success in results if success]
df_final = df_subset.loc[valid_indices].copy()

# Add the local file path column
df_final['image_path'] = df_final.index.map(lambda x: os.path.join(IMAGE_DIR, f"{x}.jpg"))

elapsed = time.time() - start_time
print(f"\n✅ Acquisition Complete in {elapsed:.2f}s")
print(f"Requested: {DOWNLOAD_LIMIT}")
print(f"Successful: {len(df_final)}")
print(f"Success Rate: {len(df_final)/DOWNLOAD_LIMIT*100:.1f}%")

# Save this valid dataset to disk so we don't have to download again if we restart
df_final.to_csv("local_training_data.csv", index=False)
print("Saved local index to 'local_training_data.csv'")

Initializing download pipeline for 2000 samples...


/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(



✅ Acquisition Complete in 10.96s
Requested: 2000
Successful: 526
Success Rate: 26.3%
Saved local index to 'local_training_data.csv'


In [23]:
# @title Cell 4: Configuration & Stratified Splitting
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, CLIPProcessor

# -----------------------------------------------------------------------------
# 1. Experimental Configuration (The "Control Center")
# -----------------------------------------------------------------------------
class Config:
    """
    Central configuration state for the experiment.
    Ensures reproducibility and ease of hyperparameter tuning.
    """
    # Data Architecture
    BATCH_SIZE = 16          # Reduced to 16 for stability on standard Colab GPUs
    IMAGE_SIZE = (224, 224)  # CLIP standard resolution
    MAX_LEN = 128            # BERT token limit

    # Model Hyperparameters
    LEARNING_RATE = 2e-5     # Standard fine-tuning rate for Transformers
    WEIGHT_DECAY = 0.01      # Regularization
    DROPOUT_RATE = 0.1
    EPOCHS = 10
    PROJECTION_DIM = 256     # Dimension of the shared fusion space
    NUM_HEADS = 4            # Attention heads in fusion layer

    # Pre-trained Assets
    BERT_MODEL = 'bert-base-uncased'
    CLIP_MODEL = 'openai/clip-vit-base-patch32'

    # Normalization Constants (OpenAI CLIP Defaults)
    CLIP_MEAN = [0.48145466, 0.4578275, 0.40821073]
    CLIP_STD = [0.26862954, 0.26130258, 0.27577711]

    # System
    SEED = 42

config = Config()

# -----------------------------------------------------------------------------
# 2. Data Partitioning
# -----------------------------------------------------------------------------
# We use the 'df_final' from Cell 3 which contains only valid, downloaded images.

print(f"Partitioning {len(df_final)} samples...")

# Split: Train (70%) vs Temp (30%)
train_df, temp_df = train_test_split(
    df_final,
    test_size=0.3,
    stratify=df_final['label'], # Critical for imbalanced datasets
    random_state=config.SEED
)

# Split Temp: Validation (15%) vs Test (15%)
val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    stratify=temp_df['label'],
    random_state=config.SEED
)

# -----------------------------------------------------------------------------
# 3. Validation & Stats
# -----------------------------------------------------------------------------
def print_split_stats(name, df):
    ratio = len(df) / len(df_final) * 100
    class_bal = df['label'].mean() * 100 # Percentage of Class 1
    print(f"   {name:<10} | {len(df):>4} samples ({ratio:>4.1f}%) | Class 1 Ratio: {class_bal:.1f}%")

print("✅ Data Splitting Complete:")
print_split_stats("Training", train_df)
print_split_stats("Validation", val_df)
print_split_stats("Test", test_df)

# -----------------------------------------------------------------------------
# 4. Processor Initialization
# -----------------------------------------------------------------------------
# We initialize these now to fail fast if internet/huggingface is down
print("\nInitializing Tokenizers...")
tokenizer = BertTokenizer.from_pretrained(config.BERT_MODEL)
print("✅ BERT Tokenizer ready.")

Partitioning 526 samples...
✅ Data Splitting Complete:
   Training   |  368 samples (70.0%) | Class 1 Ratio: 50.8%
   Validation |   79 samples (15.0%) | Class 1 Ratio: 50.6%
   Test       |   79 samples (15.0%) | Class 1 Ratio: 50.6%

Initializing Tokenizers...


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

✅ BERT Tokenizer ready.


In [27]:
# @title Cell 4.5: Upgrade to RoBERTa Tokenizer
from transformers import RobertaTokenizer

# 1. Update Config to use RoBERTa
config.BERT_MODEL = 'roberta-base'

# 2. Reload Tokenizer
print("Switching global tokenizer to RoBERTa...")
tokenizer = RobertaTokenizer.from_pretrained(config.BERT_MODEL)
print("✅ Tokenizer Updated. Now please RE-RUN CELL 5 to apply this change.")

Switching global tokenizer to RoBERTa...


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

✅ Tokenizer Updated. Now please RE-RUN CELL 5 to apply this change.


In [28]:
# @title Cell 5 (Fixed): Robust TensorFlow ETL Pipeline
import tensorflow as tf
import numpy as np

def preprocess_image(image_path):
    """
    TF Graph Operation: Reads, Decodes, Resizes, and Normalizes an image.
    Matches the specific preprocessing logic of OpenAI CLIP.
    """
    # 1. Read File
    file_content = tf.io.read_file(image_path)

    # 2. Decode Jpeg (ignore alpha channel if present, force RGB)
    img = tf.io.decode_jpeg(file_content, channels=3)

    # 3. Resize (Bicubic interpolation is standard for ViT)
    img = tf.image.resize(img, config.IMAGE_SIZE, method='bicubic')

    # 4. Rescale [0, 255] -> [0, 1]
    img = img / 255.0

    # 5. Normalize (x - mean) / std
    mean = tf.constant(config.CLIP_MEAN, dtype=tf.float32)
    std = tf.constant(config.CLIP_STD, dtype=tf.float32)
    img = (img - mean) / std

    return img

def make_dataset(dataframe, is_train=False):
    """
    Creates a highly optimized tf.data.Dataset pipeline.
    Includes strict sanitization to prevent 'Unsupported object type float' errors.
    """
    # --- SANITIZATION STEP ---
    # 1. Ensure Text is String (Replace NaNs with empty string)
    texts = dataframe['text'].fillna("").astype(str).values

    # 2. Ensure Paths are String
    paths = dataframe['image_path'].astype(str).values

    # 3. Ensure Labels are Integers (Drop NaNs if any, but fill 0 for safety here)
    labels = dataframe['label'].fillna(0).astype(np.int32).values

    # --- DATASET CREATION ---
    dataset = tf.data.Dataset.from_tensor_slices((
        {
            "text": texts,
            "image_path": paths
        },
        labels
    ))

    # 1. Shuffling (Only for training)
    if is_train:
        dataset = dataset.shuffle(buffer_size=len(dataframe))

    # 2. Mapping (Parallelized)
    def preprocess_wrapper(inputs, label):
        # Extract inputs
        text_str = inputs['text']
        img_path = inputs['image_path']

        # A. Process Image (Pure TF)
        pixel_values = preprocess_image(img_path)

        # B. Process Text (Python/HuggingFace)
        def tokenize_py(t):
            # Decode bytes to string
            t_str = t.numpy().decode('utf-8')
            enc = tokenizer(
                t_str,
                padding='max_length',
                truncation=True,
                max_length=config.MAX_LEN,
                return_token_type_ids=False, # RoBERTa doesn't use token_type_ids
                return_attention_mask=True,
                return_tensors='np'
            )
            return enc['input_ids'][0], enc['attention_mask'][0]

        input_ids, attention_mask = tf.py_function(
            func=tokenize_py,
            inp=[text_str],
            Tout=[tf.int32, tf.int32]
        )

        # Explicit shape setting required after py_function
        input_ids.set_shape([config.MAX_LEN])
        attention_mask.set_shape([config.MAX_LEN])
        pixel_values.set_shape([config.IMAGE_SIZE[0], config.IMAGE_SIZE[1], 3])

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "pixel_values": pixel_values
        }, label

    # Apply mapping with parallel processing
    dataset = dataset.map(preprocess_wrapper, num_parallel_calls=tf.data.AUTOTUNE)

    # 3. Batching & Prefetching
    dataset = dataset.batch(config.BATCH_SIZE)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset

# Initialize Pipelines
print("Constructing Data Pipelines (with sanitization)...")
train_ds = make_dataset(train_df, is_train=True)
val_ds = make_dataset(val_df, is_train=False)
test_ds = make_dataset(test_df, is_train=False)

# Verification Step
print("\nVerifying Pipeline Shapes...")
for inputs, labels in train_ds.take(1):
    print(f"   Input IDs Shape:      {inputs['input_ids'].shape}")
    print(f"   Pixel Values Shape:   {inputs['pixel_values'].shape}")
    print(f"   Labels Shape:         {labels.shape}")
    break

print("✅ Pipeline Operational.")

Constructing Data Pipelines (with sanitization)...

Verifying Pipeline Shapes...
   Input IDs Shape:      (16, 128)
   Pixel Values Shape:   (16, 224, 224, 3)
   Labels Shape:         (16,)
✅ Pipeline Operational.


In [34]:
# @title Cell 6 (Final-Final): Robust Architecture with Explicit Shapes
from transformers import TFRobertaModel, TFCLIPVisionModel
from tensorflow.keras import layers, models, regularizers
import tensorflow as tf

class CrossAttention(layers.Layer):
    """
    Novelty Layer: Text (Query) attends to Image (Key/Value).
    """
    def __init__(self, embed_dim, num_heads, dropout_rate=0.1, **kwargs):
        super(CrossAttention, self).__init__(**kwargs)
        self.mha = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, dropout=dropout_rate)
        self.ffn = models.Sequential([
            layers.Dense(embed_dim * 2, activation='gelu'),
            layers.Dense(embed_dim),
            layers.Dropout(dropout_rate)
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.add = layers.Add()

    def call(self, text_features, image_features, training=False):
        attn_output = self.mha(
            query=text_features,
            value=image_features,
            key=image_features,
            training=training
        )
        out1 = self.layernorm1(self.add([text_features, attn_output]))
        ffn_output = self.ffn(out1, training=training)
        return self.layernorm2(self.add([out1, ffn_output]))

def build_model(config):
    # 1. Inputs
    input_ids = layers.Input(shape=(config.MAX_LEN,), dtype=tf.int32, name="input_ids")
    attention_mask = layers.Input(shape=(config.MAX_LEN,), dtype=tf.int32, name="attention_mask")
    pixel_values = layers.Input(shape=(224, 224, 3), dtype=tf.float32, name="pixel_values")

    # 2. Load Encoders
    roberta = TFRobertaModel.from_pretrained(config.BERT_MODEL, use_safetensors=False)
    roberta.trainable = True

    clip = TFCLIPVisionModel.from_pretrained(config.CLIP_MODEL, use_safetensors=False)
    clip.trainable = False

    # 3. ENCODER WRAPPERS (With Explicit Output Shapes)

    def roberta_forward(inputs):
        ids, mask = inputs
        return roberta(input_ids=ids, attention_mask=mask).last_hidden_state

    def clip_forward(images):
        return clip(pixel_values=images).last_hidden_state

    # Explicitly define shapes: (Batch, Seq_Len, Dim)
    # RoBERTa base dim is 768. CLIP ViT base dim is 768.
    # Note: CLIP output sequence length is 50 (49 patches + 1 CLS)

    text_emb = layers.Lambda(
        roberta_forward,
        output_shape=(config.MAX_LEN, 768),
        name="text_encoder"
    )([input_ids, attention_mask])

    vision_emb = layers.Lambda(
        clip_forward,
        output_shape=(50, 768),
        name="vision_encoder"
    )(pixel_values)

    # 4. Projection
    text_proj = layers.Dense(config.PROJECTION_DIM)(text_emb)
    vision_proj = layers.Dense(config.PROJECTION_DIM)(vision_emb)

    # 5. Fusion (Cross Attention)
    fusion = CrossAttention(config.PROJECTION_DIM, config.NUM_HEADS)
    fused = fusion(text_proj, vision_proj)

    # 6. Classification Head
    pooled = layers.GlobalAveragePooling1D()(fused)
    x = layers.Dense(64, activation='gelu', kernel_regularizer=regularizers.l2(1e-4))(pooled)
    x = layers.Dropout(config.DROPOUT_RATE)(x)
    output = layers.Dense(1, activation='sigmoid', name="output")(x)

    return models.Model(inputs=[input_ids, attention_mask, pixel_values], outputs=output)

# Compile
try:
    strategy = tf.distribute.MirroredStrategy()
except:
    strategy = tf.distribute.get_strategy()

with strategy.scope():
    print("Building model with Lambda wrappers and explicit shapes...")
    model = build_model(config)

    model.compile(
        optimizer=tf.keras.optimizers.AdamW(learning_rate=config.LEARNING_RATE, weight_decay=config.WEIGHT_DECAY),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )

model.summary()

Building model with Lambda wrappers and explicit shapes...


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.
Some layers from the model checkpoint at openai/clip-vit-base-patch32 were not used when initializing TFCLIPVisionModel: ['clip/text_model/encoder/layers_._0/layer_norm2/bet

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_ids           │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_mask      │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pixel_values        │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ text_encoder        │ (None, 128, 768)  │          0 │ input_ids[0][0],  │
│ (Lambda)            │                   │            │ attention_mask[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ vision_encoder      │ (None, 50, 768)   │          0 │ pixel_values[0][… │
│ (Lambda)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128, 256)  │    196,864 │ text_encoder[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 50, 256)   │    196,864 │ vision_encoder[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cross_attention     │ (None, 128, 256)  │  1,315,840 │ dense[0][0],      │
│ (CrossAttention)    │                   │            │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 256)       │          0 │ cross_attention[… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 64)        │     16,448 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 64)        │          0 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 1)         │         65 │ dropout_2[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,726,081 (6.58 MB)

 Trainable params: 1,726,081 (6.58 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# @title Cell 6 & 7: Architecture, Build, and Train
from transformers import TFRobertaModel, TFCLIPVisionModel
from tensorflow.keras import layers, models, regularizers
import tensorflow as tf
import numpy as np

# 1. Define Cross-Attention Layer
class CrossAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads, dropout_rate=0.1, **kwargs):
        super(CrossAttention, self).__init__(**kwargs)
        self.mha = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, dropout=dropout_rate)
        self.ffn = models.Sequential([
            layers.Dense(embed_dim * 2, activation='gelu'),
            layers.Dense(embed_dim),
            layers.Dropout(dropout_rate)
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.add = layers.Add()

    def call(self, text_features, image_features, training=False):
        attn_output = self.mha(
            query=text_features,
            value=image_features,
            key=image_features,
            training=training
        )
        out1 = self.layernorm1(self.add([text_features, attn_output]))
        ffn_output = self.ffn(out1, training=training)
        return self.layernorm2(self.add([out1, ffn_output]))

# 2. Model Builder with Wrappers
def build_model(config):
    # Inputs
    input_ids = layers.Input(shape=(config.MAX_LEN,), dtype=tf.int32, name="input_ids")
    attention_mask = layers.Input(shape=(config.MAX_LEN,), dtype=tf.int32, name="attention_mask")
    pixel_values = layers.Input(shape=(224, 224, 3), dtype=tf.float32, name="pixel_values")

    # Load Encoders
    roberta = TFRobertaModel.from_pretrained(config.BERT_MODEL, use_safetensors=False)
    roberta.trainable = True

    clip = TFCLIPVisionModel.from_pretrained(config.CLIP_MODEL, use_safetensors=False)
    clip.trainable = False

    # Wrapper Functions
    def roberta_forward(inputs):
        ids, mask = inputs
        return roberta(input_ids=ids, attention_mask=mask).last_hidden_state

    def clip_forward(images):
        # FIX: Transpose from (Batch, 224, 224, 3) to (Batch, 3, 224, 224)
        # This satisfies the HF model's internal channel-first expectation
        images = tf.transpose(images, perm=[0, 3, 1, 2])
        return clip(pixel_values=images).last_hidden_state

    # Apply Wrappers with Explicit Shapes
    text_emb = layers.Lambda(
        roberta_forward,
        output_shape=(config.MAX_LEN, 768),
        name="text_encoder"
    )([input_ids, attention_mask])

    vision_emb = layers.Lambda(
        clip_forward,
        output_shape=(50, 768),
        name="vision_encoder"
    )(pixel_values)

    # Projection & Fusion
    text_proj = layers.Dense(config.PROJECTION_DIM)(text_emb)
    vision_proj = layers.Dense(config.PROJECTION_DIM)(vision_emb)

    fusion = CrossAttention(config.PROJECTION_DIM, config.NUM_HEADS)
    fused = fusion(text_proj, vision_proj)

    # Classification
    pooled = layers.GlobalAveragePooling1D()(fused)
    x = layers.Dense(64, activation='gelu', kernel_regularizer=regularizers.l2(1e-4))(pooled)
    x = layers.Dropout(config.DROPOUT_RATE)(x)
    output = layers.Dense(1, activation='sigmoid', name="output")(x)

    return models.Model(inputs=[input_ids, attention_mask, pixel_values], outputs=output)

# 3. Build & Compile
try:
    strategy = tf.distribute.MirroredStrategy()
except:
    strategy = tf.distribute.get_strategy()

with strategy.scope():
    print("Constructing model...")
    model = build_model(config)

    # --- WARMUP ---
    print("Initializing variables (Warmup)...")
    dummy_ids = tf.zeros((1, config.MAX_LEN), dtype=tf.int32)
    dummy_mask = tf.zeros((1, config.MAX_LEN), dtype=tf.int32)
    dummy_pixels = tf.random.normal((1, 224, 224, 3))

    model([dummy_ids, dummy_mask, dummy_pixels])
    print("✅ Model Built Successfully.")

    model.compile(
        optimizer=tf.keras.optimizers.AdamW(learning_rate=config.LEARNING_RATE, weight_decay=config.WEIGHT_DECAY),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )

model.summary()

# 4. Train
print(f"\nStarting training for {config.EPOCHS} epochs...")
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

callbacks = [
    ModelCheckpoint("best_multimodal_model.h5", save_best_only=True, monitor="val_auc", mode="max"),
    EarlyStopping(monitor="val_auc", patience=3, mode="max", restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=1)
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=config.EPOCHS,
    callbacks=callbacks
)

print("✅ Training Complete.")

Constructing model...


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.
Some layers from the model checkpoint at openai/clip-vit-base-patch32 were not used when initializing TFCLIPVisionModel: ['clip/text_model/encoder/layers_._0/layer_norm2/bet

Initializing variables (Warmup)...
✅ Model Built Successfully.


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_ids           │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_mask      │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pixel_values        │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ text_encoder        │ (None, 128, 768)  │          0 │ input_ids[0][0],  │
│ (Lambda)            │                   │            │ attention_mask[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ vision_encoder      │ (None, 50, 768)   │          0 │ pixel_values[0][… │
│ (Lambda)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 128, 256)  │    196,864 │ text_encoder[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 50, 256)   │    196,864 │ vision_encoder[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cross_attention_2   │ (None, 128, 256)  │  1,315,840 │ dense_10[0][0],   │
│ (CrossAttention)    │                   │            │ dense_11[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 256)       │          0 │ cross_attention_… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_14 (Dense)    │ (None, 64)        │     16,448 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_8 (Dropout) │ (None, 64)        │          0 │ dense_14[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 1)         │         65 │ dropout_8[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

...

23/23 ━━━━━━━━━━━━━━━━━━━━ 331s 14s/step - accuracy: 0.5999 - auc: 0.6878 - loss: 0.6528 - val_accuracy: 0.6709 - val_auc: 0.7038 - val_loss: 0.6440 - learning_rate: 2.0000e-05
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.6074 - auc: 0.7021 - loss: 0.6395 

23/23 ━━━━━━━━━━━━━━━━━━━━ 331s 14s/step - accuracy: 0.6097 - auc: 0.7032 - loss: 0.6388 - val_accuracy: 0.7089 - val_auc: 0.7369 - val_loss: 0.6141 - learning_rate: 2.0000e-05
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.7014 - auc: 0.7736 - loss: 0.5919 

23/23 ━━━━━━━━━━━━━━━━━━━━ 331s 14s/step - accuracy: 0.7010 - auc: 0.7740 - loss: 0.5917 - val_accuracy: 0.6962 - val_auc: 0.7420 - val_loss: 0.5931 - learning_rate: 2.0000e-05
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.7985 - auc: 0.8491 - loss: 0.5334 

23/23 ━━━━━━━━━━━━━━━━━━━━ 331s 14s/step - accuracy: 0.7977 - auc: 0.8483 - loss: 0.5336 - val_accuracy: 0.7342 - val_auc: 0.7474 - val_loss: 0.5730 - learning_rate: 2.0000e-05
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.7413 - auc: 0.8247 - loss: 0.5329 

23/23 ━━━━━━━━━━━━━━━━━━━━ 330s 14s/step - accuracy: 0.7423 - auc: 0.8258 - loss: 0.5320 - val_accuracy: 0.6076 - val_auc: 0.7532 - val_loss: 0.5999 - learning_rate: 2.0000e-05
Epoch 7/10
18/23 ━━━━━━━━━━━━━━━━━━━━ 59s 12s/step - accuracy: 0.7333 - auc: 0.8512 - loss: 0.5163 